<a href="https://colab.research.google.com/github/monksealseal/TropicalCycloneML/blob/main/Worldsphere_Backbone120622.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import xarray as xr
import tensorflow as tf
import fastai
import pytorch
import wandb
import gradio

from timm import create_model

# Set the bucket name for the images
BUCKET_NAME = 'hurricane_images'

# Set the netCDF file and image dimensions
NETCDF_FILE = 'hurricane_data.nc'
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

# Load the netCDF file using xarray
ds = xr.open_dataset(NETCDF_FILE)

# Convert the data to a 3D array of images
data = ds.to_array().transpose('time', 'x', 'y')
images = []
for i in range(data.shape[0]):
    image = data[i].values
    image = image / image.max()
    image = (image * 255).astype(np.uint8)
    image = image.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, 1))
    images.append(image)

# Store the images in a new bucket in Google Cloud Storage
bucket = storage.Client().bucket(BUCKET_NAME)
for i, image in enumerate(images):
    blob = bucket.blob('hurricane_{}.png'.format(i))
    blob.upload_from_string(image.tobytes())

# Load the training data and labels
data = np.load('hurricane_data.npy')
X_train = data['X']
y_train = data['y']

# Use fast.ai to try out all the models in the timm library
models = create_model('tf_efficientnet_b1', pretrained=True)
for model in models:
    learner = fastai.Learner(model, data)
    learner.fit_one_cycle(10)
    wandb.log({'fast.ai_loss': learner.recorder.losses[-1]})

# Use PyTorch to try out all the models in the timm library
models = create_model('tf_efficientnet_b1', pretrained=True)
for model in models:
    model = model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.BCEWithLogitsLoss()
    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for i, (x, y) in enumerate(zip(X_train, y_train)):
            x = x.to('cuda')
            y = y.to('cuda')
            outputs = model(x)
            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer


ModuleNotFoundError: ignored

In [2]:
# Use fast.ai to try out all the models in the timm library
models = create_model('tf_efficientnet_b1', pretrained=True)
for model in models:
    learner = fastai.Learner(model, data)
    learner.fit_one_cycle(10)
    wandb.log({'fast.ai_loss': learner.recorder.losses[-1]})

# Use PyTorch to try out all the models in the timm library
models = create_model('tf_efficientnet_b1', pretrained=True)
for model in models:
    model = model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.BCEWithLogitsLoss()
    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for i, (x, y) in enumerate(zip(X_train, y_train)):
            x = x.to('cuda')
            y = y.to('cuda')
            outputs = model(x)
            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        wandb.log({'pytorch_loss': running_loss / len(X_train)})


NameError: ignored

In [3]:
# Use wandb to compare the performance of the different models
results = wandb.Chart(data=wandb.run.summary).data()
fast_ai_results = results['fast.ai_loss']
pytorch_results = results['pytorch_loss']
best_model = min(fast_ai_results, key=fast_ai_results.get)

# Use gradio to deploy the best model to huggingface spaces
model = best_model.model
model.to('cuda')
gr = gradio.Interface(model, inputs="images", outputs="predictions")
gr.launch()


NameError: ignored